In [26]:
!pip cache purge
!pip install --upgrade pip  # Upgrade pip to the latest version
!pip install huggingface_hub  # Install using the new package name
!pip install huggingface-cli
!pip install transformers accelerate bitsandbytes
!pip install torch torchvision torchaudio
!pip install tiktoken
!pip install transformers_stream_generator # Install the missing package
!huggingface-cli login

Files removed: 2
ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a tok

In [27]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, pipeline
import torch
from google.colab import userdata



In [28]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", trust_remote_code=True)


messages = [
    {"role": "system", "content": "eres un asistente util"},
    {"role": "user", "content":"Cuentame un chiste"}
]

# Define a chat template
chat_template = """{% for message in messages %}\
{{message['role']}}: {{message['content']}}\
{% endfor %}\
"""

# Apply the chat template
prompt = tokenizer.apply_chat_template(
    messages,
    chat_template=chat_template,  # Pass the chat template as an argument
    tokenize=True,
    add_generation_prompt=True
)
print(prompt)

[2291, 1024, 9413, 2229, 4895, 2004, 27870, 10111, 21183, 4014, 20330, 1024, 16091, 12380, 4168, 4895, 9610, 13473]


In [29]:


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", trust_remote_code=True)

messages = [
    {"role": "system", "content": " eres un asistente util "},
    {"role": "user", "content":" Cuentame un chiste "}
]

# Define a chat template
chat_template = """{% for message in messages %}\
{{message['role']}}: {{message['content']}}\
{% endfor %}\
"""

# Apply the chat template
prompt = tokenizer.apply_chat_template(
    messages,
    chat_template=chat_template,  # Pass the chat template as an argument
    tokenize=False,
    add_generation_prompt=True
)
print(prompt)

system:  eres un asistente util user:  Cuentame un chiste 


In [30]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased", trust_remote_code=True)

messages = [
    {"role": "system", "content": "eres un asistente util"},
    {"role": "user", "content":"Cuentame un chiste "}
]

# Define a chat template
chat_template = """{% for message in messages %}\
{{message['role']}}: {{message['content']}}\
{% endfor %}\
"""

# Apply the chat template
prompt = tokenizer.apply_chat_template(
    messages,
    chat_template=chat_template,  # Pass the chat template as an argument
    tokenize=True,
    add_generation_prompt=True
)

# Convert the prompt to a PyTorch tensor
# prompt is a list of token ids, convert it to a tensor of shape (1, sequence_length)
# and move it to the device the model is on (if using GPU)
prompt_tensor = torch.tensor([prompt]) #.to(model.device)

# Now pass the prompt_tensor to the model
# assuming you want the last hidden state
outputs = model(prompt_tensor)
last_hidden_state = outputs.last_hidden_state
print(last_hidden_state)
#print(model.batch_decode(prompt)) #this also requires a tensor input
#print(prompt) #this is fine

tensor([[[-0.2169, -0.0335,  0.2140,  ..., -0.1135,  0.4125,  0.5910],
         [-0.1134,  0.1481,  0.2694,  ..., -0.2174,  0.3267,  0.5415],
         [-0.0520,  0.0321,  0.3448,  ..., -0.2631,  0.3581,  0.8714],
         ...,
         [-0.5088,  0.2732,  0.3107,  ...,  0.0639, -0.1173,  1.1314],
         [-0.2840, -0.2624,  0.3160,  ..., -0.1986, -0.0248,  0.9651],
         [-0.5643, -0.1085,  0.3658,  ..., -0.1432, -0.0546,  0.7710]]],
       grad_fn=<NativeLayerNormBackward0>)


In [35]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B", trust_remote_code=True)

text = "Hola amigos"

print(tokenizer.tokenize(text))
print()
print(tokenizer.encode(text))
print()
print(tokenizer.decode(tokenizer.encode(text)))

messages = [
    {"role": "system", "content": "eres un asistente util"},
    {"role": "user", "content": "Cuentame un chiste"}
]

# Qwen typically uses a specific format for multi-turn conversations.
# You might need to adapt this based on the specific Qwen model's documentation.
# This is a common format:
chat_template = """{% for message in messages %}
{% if message['role'] == 'system' %}<|im_start|>system
{{message['content']}}<|im_end|>
{% elif message['role'] == 'user' %}<|im_start|>user
{{message['content']}}<|im_end|>
{% elif message['role'] == 'assistant' %}<|im_start|>assistant
{{message['content']}}<|im_end|>
{% endif %}
{% endfor %}"""

prompt = tokenizer.apply_chat_template(
    messages,
    chat_template=chat_template,
    tokenize=False,
    add_generation_prompt=False  # Qwen usually doesn't need an explicit generation prompt here
)


print(prompt)

['Hola', 'Ġamigos']

[68012, 56557]

Hola amigos
<|im_start|>system
eres un asistente util<|im_end|>
<|im_start|>user
Cuentame un chiste<|im_end|>

